# Scraping Financial Data from Macrotrends.net

- [Disclaimer](#info)
- [Importing libraries](#imports)
- [Creating the empty excels](#excel)
- [Scraping the data](#pl)
    - [Profit & Loss Statements](#pl)
    - [Balance Sheet](#bs)
    - [Cash Flow Statements](#cf)
    - [Ratios](#ratios)
    - [Stocks data](#stocks)    
- [Next Steps](#steps)

<a id=info></a>

<div class="alert alert-info">
    <H2>Disclaimer:</H2>
<ul>
    <li>This analysis was done for learning purposes (not commercial). I am learning Python and wanted to learn webscraping</li>
    <li>This notebook in particular is quite old, I created it 1 year ago, only 1 month after starting to learn Python
    <li>Therefore, there are many inefficiencies that I haven't corrected yet
    <li>The good thing is it still works! I have just tried it with recently launched version of Python 3.9.0</li>
    <li>I am uploading it just now beacuse I want to keep track of it for future use</li>
</ul>
</div>

<a id=imports></a>

## Imports

In [22]:
import re
import json
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

<a id=excel></a>

## Creating Empty Excels

In [18]:
# Find the company you want information from and copy paste the url of the first page in macrotrends (stock-price-history)

url = ['https://www.macrotrends.net/stocks/charts/FB/facebook/stock-price-history',
       'https://www.macrotrends.net/stocks/charts/AMZN/amazon/stock-price-history',
       'https://www.macrotrends.net/stocks/charts/AAPL/apple/stock-price-history',
       'https://www.macrotrends.net/stocks/charts/NFLX/netflix/stock-price-history', 
       'https://www.macrotrends.net/stocks/charts/GOOGL/alphabet/stock-price-history']

In [19]:
# This part generates useful URL variables for the following code

companies = []
url_code = []

for org in url:
    url_section = org.split('/')[5:7]
    ticker = url_section[0]
    name = url_section[1]
    url_section = ticker+'/'+name
    
    companies.append(name)
    url_code.append(url_section)
    print(url_section)

FB/facebook
AMZN/amazon
AAPL/apple
NFLX/netflix
GOOGL/alphabet


In [21]:
# This part creates a new empty excel document in format XLSM (you can change it if you want other formats)
# Note: for some reason I have not found out yet, you sometimes have to run this cell several times for it to work properly

import openpyxl

for company in companies:
    file = 'excels/'+company+'.xlsm'
    wb = openpyxl.Workbook()
    ws = wb.active
    wb.save(file)
    wb = openpyxl.load_workbook(file, keep_vba=True)
    wb.save(file)

<a id=pl></a>

## SCRAPING

## Profit and Loss Statement

NOTE: All the code is set to provide information at Quarterly Level. Macrotrends also allows to generate data at annual level. To do it erase "?freq=Q" from all the urls

In [23]:
for company in url_code:
    r = requests.get('https://www.macrotrends.net/stocks/charts/'+company+'/income-statement?freq=Q')
    p = re.compile(r' var originalData = (.*?);\r\n\r\n\r',re.DOTALL)
    data = json.loads(p.findall(r.text)[0])
    headers = list(data[0].keys())
    headers.remove('popup_icon')
    result = []

    for row in data:
        soup = bs(row['field_name'])
        field_name = soup.select_one('a, span').text
        fields = list(row.values())[2:]
        fields.insert(0, field_name)
        result.append(fields)

    pd.option_context('display.max_rows', None, 'display.max_columns', None)
    df = pd.DataFrame(result, columns = headers)
    
    
    df.set_index('field_name', inplace=True)
    
    for i in df.columns:
        df[i] = pd.to_numeric(df[i],errors='coerce')
    
    
    name = company.split('/')
    
    writer = pd.ExcelWriter('excels/'+name[1]+'.xlsm', engine='openpyxl')
    writer.book = openpyxl.load_workbook('excels/'+name[1]+'.xlsm', keep_vba= True)
    writer.sheets = {ws.title: ws for ws in writer.book.worksheets}
    df.to_excel(writer, sheet_name='ProfitLoss')
    workbook = writer.book
    workbook.filename = 'excels/'+name[1]+'.xlsm'
    writer.save()
    writer.close()
    

<a id=bs></a>

## Balance Sheet

In [24]:
for company in url_code:
    r = requests.get('https://www.macrotrends.net/stocks/charts/'+company+'/balance-sheet?freq=Q')
    p = re.compile(r' var originalData = (.*?);\r\n\r\n\r',re.DOTALL)
    data = json.loads(p.findall(r.text)[0])
    headers = list(data[0].keys())
    headers.remove('popup_icon')
    result = []

    for row in data:
        soup = bs(row['field_name'])
        field_name = soup.select_one('a, span').text
        fields = list(row.values())[2:]
        fields.insert(0, field_name)
        result.append(fields)

    pd.option_context('display.max_rows', None, 'display.max_columns', None)
    df = pd.DataFrame(result, columns = headers)

    df.set_index('field_name', inplace=True)
    
    for i in df.columns:
        df[i] = pd.to_numeric(df[i],errors='coerce')
        
    name = company.split('/')
    
    writer = pd.ExcelWriter('excels/'+name[1]+'.xlsm', engine='openpyxl')
    writer.book = openpyxl.load_workbook('excels/'+name[1]+'.xlsm', keep_vba= True)
    writer.sheets = {ws.title: ws for ws in writer.book.worksheets}
    df.to_excel(writer, sheet_name='BalanceSheet')
    workbook = writer.book
    workbook.filename = 'excels/'+name[1]+'.xlsm'
    writer.save()
    writer.close()

<a id=cf></a>

## Cashflow Statement

In [25]:
for company in url_code:
    r = requests.get('https://www.macrotrends.net/stocks/charts/'+company+'/cash-flow-statement?freq=Q')
    p = re.compile(r' var originalData = (.*?);\r\n\r\n\r',re.DOTALL)
    data = json.loads(p.findall(r.text)[0])
    headers = list(data[0].keys())
    headers.remove('popup_icon')
    result = []

    for row in data:
        soup = bs(row['field_name'])
        field_name = soup.select_one('a, span').text
        fields = list(row.values())[2:]
        fields.insert(0, field_name)
        result.append(fields)

    pd.option_context('display.max_rows', None, 'display.max_columns', None)
    df = pd.DataFrame(result, columns = headers)

    df.set_index('field_name', inplace=True)
    
    for i in df.columns:
        df[i] = pd.to_numeric(df[i],errors='coerce')
        
    name = company.split('/')
    
    writer = pd.ExcelWriter('excels/'+name[1]+'.xlsm', engine='openpyxl')
    writer.book = openpyxl.load_workbook('excels/'+name[1]+'.xlsm', keep_vba= True)
    writer.sheets = {ws.title: ws for ws in writer.book.worksheets}
    df.to_excel(writer, sheet_name='CashFlow')
    workbook = writer.book
    workbook.filename = 'excels/'+name[1]+'.xlsm'
    writer.save()
    writer.close()

<a id=ratios></a>

## Financial Ratios

In [26]:
for company in url_code:
    r = requests.get('https://www.macrotrends.net/stocks/charts/'+company+'/financial-ratios?freq=A')
    p = re.compile(r' var originalData = (.*?);\r\n\r\n\r',re.DOTALL)
    data = json.loads(p.findall(r.text)[0])
    headers = list(data[0].keys())
    headers.remove('popup_icon')
    result = []

    for row in data:
        soup = bs(row['field_name'])
        field_name = soup.select_one('a, span').text
        fields = list(row.values())[2:]
        fields.insert(0, field_name)
        result.append(fields)

    pd.option_context('display.max_rows', None, 'display.max_columns', None)
    df = pd.DataFrame(result, columns = headers)

    df.set_index('field_name', inplace=True)
    
    for i in df.columns:
        df[i] = pd.to_numeric(df[i],errors='coerce')
        
    name = company.split('/')
    
    writer = pd.ExcelWriter('excels/'+name[1]+'.xlsm', engine='openpyxl')
    writer.book = openpyxl.load_workbook('excels/'+name[1]+'.xlsm', keep_vba= True)
    writer.sheets = {ws.title: ws for ws in writer.book.worksheets}
    df.to_excel(writer, sheet_name='Ratios')
    workbook = writer.book
    workbook.filename = 'excels/'+name[1]+'.xlsm'
    writer.save()
    writer.close()

<a id=stocks></a>

## Stocks Data

In [27]:
for company in url_code:
    stocks = pd.read_html('https://www.macrotrends.net/stocks/charts/'+company+'/stock-price-history', attrs={'class':'historical_data_table table'})
    stocks = stocks[0]
    stocks.columns = stocks.columns.droplevel()
    stocks.set_index('Year')
    
    name = company.split('/')
    
    writer = pd.ExcelWriter('excels/'+name[1]+'.xlsm', engine='openpyxl')
    writer.book = openpyxl.load_workbook('excels/'+name[1]+'.xlsm', keep_vba= True)
    writer.sheets = {ws.title: ws for ws in writer.book.worksheets}
    stocks.to_excel(writer, sheet_name='StockPrice')
    workbook = writer.book
    workbook.remove(workbook['Sheet'])
    workbook.filename = 'excels/'+name[1]+'.xlsm'
    writer.save()
    writer.close()

<a id=steps></a>

## Next Steps

Currently this code is not very efficient, I am thinking on changing some things like creating classes and reducing the number of for loops in the code. 